In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix  # Import required metrics
import seaborn as sns
import os
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model

In [3]:
#Convert image to unsigned integer
def convert_to_uint8(image):
    """
    Convert the image to uint8.
    """
    image = image * 255.0  # Assuming the image is in the range [0, 1]
    image = tf.cast(image, tf.uint8)
    return image


# Define image data generator
train_datagen = ImageDataGenerator(
    rescale=1./255,       # Rescale pixel values from 0-255 to 0-1
# preprocessing_function=convert_to_uint8,  # Apply the custom preprocessing function
    shear_range=0.2,      # Randomly apply shearing transformation
    zoom_range=0.2,       # Randomly zoom in images
    horizontal_flip=True, # Randomly flip images horizontally
    validation_split=0.2  # Split the data into training and validation sets
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    r'C:\Users\Boluwatife\Steganalysis\dataset\train',  # Path to the dataset directory
    target_size=(256, 256),   # Resize images to 256x256 pixels
    batch_size=32,            # Number of images to yield per batch
    class_mode='binary',      # Binary labels (use 'categorical' for multi-class)
    subset='training'         # Use the training subset
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
     r'C:\Users\Boluwatife\Steganalysis\dataset\val',  # Use the same directory as above
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation'       # Use the validation subset
)


Found 5117 images belonging to 2 classes.
Found 319 images belonging to 2 classes.


In [16]:
# Load Images Using ImageDataGenerator
def load_images(data_dir, target_size=(128, 128), batch_size=32):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        data_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',  # 'binary' or 'categorical' based on your dataset
        color_mode='grayscale',  # Using grayscale for GLCM
        shuffle=True
    )
    return generator

# Apply GLCM to Extract Features
def apply_glcm(image):
    glcm = graycomatrix(image, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast').flatten()
    dissimilarity = graycoprops(glcm, 'dissimilarity').flatten()
    homogeneity = graycoprops(glcm, 'homogeneity').flatten()
    energy = graycoprops(glcm, 'energy').flatten()
    correlation = graycoprops(glcm, 'correlation').flatten()
    asm = graycoprops(glcm, 'ASM').flatten()
    return np.concatenate([contrast, dissimilarity, homogeneity, energy, correlation, asm])

# Save Processed Images
def save_image(image, save_dir, image_name):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, image_name)
    cv2.imwrite(save_path, image * 255)  # Re-multiply by 255 if originally normalized

# Example Workflow
def process_images(data_dir, output_dir, batch_size=32, target_size=(128, 128)):
    generator = load_images(data_dir, target_size=target_size, batch_size=batch_size)
    
    all_glcm_features = []
    all_labels = []
    
    # Get class labels from the generator
    class_labels = {v: k for k, v in generator.class_indices.items()}

    for i, (X_batch, y_batch) in enumerate(generator):
        if i >= len(generator):  # Avoid infinite loop
            break
        
        for j in range(X_batch.shape[0]):
            image = X_batch[j].squeeze()  # Grayscale image
            label = y_batch[j]
            
            # Apply GLCM feature extraction
            glcm_features = apply_glcm(image.astype(np.uint8))
            all_glcm_features.append(glcm_features)
            all_labels.append(label)
    
    # Convert to NumPy arrays
    X_glcm = np.array(all_glcm_features)
    y_glcm = np.array(all_labels)
    
    
    # Reshape GLCM features back to image format and save the processed images
    for i, (glcm_feature, label) in enumerate(zip(X_glcm, y_glcm)):
        reshaped_image = glcm_feature.reshape((4, 6))  # Adjusted shape to match the size of 24
        image_name = f"image_{i}.png"
        class_dir = class_labels[label]
        save_image(reshaped_image, os.path.join(output_dir, class_dir), image_name)


# Directory paths
data_dir = r'C:\Users\Boluwatife\Steganalysis\dataset_original'  # Replace with your dataset path
output_dir = r'C:\Users\Boluwatife\Steganalysis\glcm_dataset'  # Where processed images will be saved

# Process the images
process_images(data_dir, output_dir)


Found 7995 images belonging to 2 classes.


In [17]:
def split_dataset(data_dir, output_dir, val_split=0.2):
    """
    Splits dataset into training and validation directories.

    Parameters:
    - data_dir: Directory containing the original dataset.
    - output_dir: Directory where the train and val subdirectories will be created.
    - val_split: Fraction of images to be used for validation.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Get the class directories
    class_dirs = [os.path.join(data_dir, d) for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
    
    for class_dir in class_dirs:
        class_name = os.path.basename(class_dir)
        # Create train and val directories
        train_dir = os.path.join(output_dir, 'train', class_name)
        val_dir = os.path.join(output_dir, 'val', class_name)
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(val_dir, exist_ok=True)
        
        # List all images in the class directory
        images = [os.path.join(class_dir, f) for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
        
        # Split images into train and val sets
        train_files, val_files = train_test_split(images, test_size=val_split, random_state=42)
        
        # Move files to corresponding directories
        for file in train_files:
            shutil.copy(file, train_dir)
        for file in val_files:
            shutil.copy(file, val_dir)

    print(f"Dataset split into 'train' and 'val' directories under {output_dir}")

# Directory paths
data_dir = r'C:\Users\Boluwatife\Steganalysis\glcm_dataset'  # Replace with your dataset path
output_dir = r'C:\Users\Boluwatife\Steganalysis\split_glcm_dataset'  # Directory to store the train and val directories

# Split the dataset
split_dataset(data_dir, output_dir)


Dataset split into 'train' and 'val' directories under C:\Users\Boluwatife\Steganalysis\split_glcm_dataset


In [5]:
#Use ImageGenerator to load and preprocess image

def create_generators(train_dir, val_dir, target_size=(128, 128), batch_size=32):
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',  # or 'categorical'
        color_mode='grayscale',  # Assuming grayscale images
        shuffle=True
    )
    
    val_generator = datagen.flow_from_directory(
        val_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',  # or 'categorical'
        color_mode='grayscale',  # Assuming grayscale images
        shuffle=False
    )
    
    return train_generator, val_generator

train_dir, val_dir = r'C:\Users\Boluwatife\Steganalysis\split_glcm_dataset\train', r'C:\Users\Boluwatife\Steganalysis\split_glcm_dataset\val' 

train_generator, val_generator = create_generators(train_dir, val_dir)

Found 6396 images belonging to 2 classes.
Found 1599 images belonging to 2 classes.


In [22]:
def build_srnet(input_shape=(256, 256, 1)):
    model = models.Sequential()

    # Add an explicit Input layer
    model.add(layers.Input(shape=input_shape))

    # Block 1
    model.add(layers.Conv2D(64, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2, 2)))

    # Block 2
    model.add(layers.Conv2D(128, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2, 2)))

    # Block 3
    model.add(layers.Conv2D(256, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.MaxPooling2D((2, 2)))

    # Block 4
    model.add(layers.Conv2D(512, (3, 3), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.GlobalAveragePooling2D())

    # Dense Layers
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))

    # Output Layer (Binary classification for steganalysis)
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

# Example usage
input_shape = (256, 256, 1)  # Assuming grayscale images of size 256x256
model = build_srnet(input_shape)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_20 (Conv2D)                   │ (None, 256, 256, 64)        │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_30               │ (None, 256, 256, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_20 (ReLU)                      │ (None, 256, 256, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_15 (MaxPooling2D)      │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_21 (Conv2D)                   │ (None, 128, 128, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_31               │ (None, 128, 128, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_21 (ReLU)                      │ (None, 128, 128, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_16 (MaxPooling2D)      │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_22 (Conv2D)                   │ (None, 64, 64, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_32               │ (None, 64, 64, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_22 (ReLU)                      │ (None, 64, 64, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_17 (MaxPooling2D)      │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_23 (Conv2D)                   │ (None, 32, 32, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_33               │ (None, 32, 32, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_23 (ReLU)                      │ (None, 32, 32, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_5           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_34               │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 1,719,553 (6.56 MB)

 Trainable params: 1,716,865 (6.55 MB)

 Non-trainable params: 2,688 (10.50 KB)

In [3]:
# Define image size and batch size
img_height, img_width = 256, 256
batch_size = 32  # Make sure to define batch_size

In [27]:
# Train the model using fit_generator

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size
)

Epoch 1/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 338s 2s/step - accuracy: 0.5127 - loss: 0.8840 - val_accuracy: 0.5102 - val_loss: 0.7305
Epoch 2/30
  1/199 ━━━━━━━━━━━━━━━━━━━━ 5:26 2s/step - accuracy: 0.5312 - loss: 0.7789

C:\Users\Boluwatife\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


199/199 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5312 - loss: 0.7789 - val_accuracy: 0.0000e+00 - val_loss: 1.0341
Epoch 3/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 339s 2s/step - accuracy: 0.5105 - loss: 0.7848 - val_accuracy: 0.4898 - val_loss: 0.7276
Epoch 4/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5938 - loss: 0.7296 - val_accuracy: 1.0000 - val_loss: 0.4667
Epoch 5/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 340s 2s/step - accuracy: 0.5209 - loss: 0.7289 - val_accuracy: 0.4898 - val_loss: 0.7053
Epoch 6/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4062 - loss: 0.6926 - val_accuracy: 1.0000 - val_loss: 0.5949
Epoch 7/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 343s 2s/step - accuracy: 0.5274 - loss: 0.7053 - val_accuracy: 0.5102 - val_loss: 0.8327
Epoch 8/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5000 - loss: 0.6785 - val_accuracy: 0.0000e+00 - val_loss: 1.3853
Epoch 9/30
199/199 ━━━━━━━━━━━━━━━━━━━━ 344s 2s/step - accuracy: 0.5188 - loss: 0.7075 - val_accuracy: 0

ValueError: Found input variables with inconsistent numbers of samples: [1599, 1568]

In [ ]:
# Save the model
model.save('glcm.keras')

In [6]:
# Load your saved Keras model
model = load_model(r'C:\Users\Boluwatife\Steganalysis\glcm_model.keras')

C:\Users\Boluwatife\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 28 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [7]:
# Evaluate the model

from keras.models import load_model
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Assuming you have already set up the ImageDataGenerator for the test data
# Example for test generator setup

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the test data with the image generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    r'C:\Users\Boluwatife\Steganalysis\test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='categorical',  # or 'binary' if it's binary classification
    shuffle=False  # Ensure that shuffling is disabled for proper evaluation
)

Found 4000 images belonging to 2 classes.


In [8]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Get model predictions
y_pred_prob = model.predict(test_generator)
y_pred = (y_pred_prob > 0.5).astype(int)  # Convert probabilities to binary predictions

# Get true labels
y_true = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Evaluate metrics
accuracy = np.sum(y_true == y_pred.flatten()) / len(y_true)

# Print classification report and confusion matrix
print(f"Accuracy: {accuracy:.4f}")

print("Classification Report:")
print(classification_report(y_true, y_pred.flatten(), target_names=class_labels))

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred.flatten()))

C:\Users\Boluwatife\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step
Accuracy: 0.4943
Classification Report:
              precision    recall  f1-score   support

   cleanTest       0.50      0.63      0.55      2000
   stegoTest       0.49      0.36      0.42      2000

    accuracy                           0.49      4000
   macro avg       0.49      0.49      0.49      4000
weighted avg       0.49      0.49      0.49      4000

Confusion Matrix:
[[1251  749]
 [1274  726]]
